# Adaptive sampling

This script shows how to get geomtries out of the files produced with SchNarc using the adaptive sampling mode, an example to run the calculations with SHARC and parser to get the data out of SHARC-output files including a function to extend the data base.

In [21]:
path = "./"
import ase.io
import ase
import numpy as np
import schnetpack as spk
from ase.units import Bohr
print(Bohr)
import os
SHARC="/user/julia/software/SchNarc/sharc/source/../bin/"

0.5291772105638411


In [22]:
def read_QMout(data,geoms,nfiles,natoms,nnacs,ndipoles,nstates,path,soc_flag,nsinglets,ntriplets):
    dataiterator=-1
    atoms=[]
    for i in range(nfiles):
        try:
            file = open("%s_%05d/QM.out"%(path,i), "r").readlines()
            skip=False
        except IOError:
            skip=True
        if skip == False:
            dataiterator+=1
            #each data point gets one entry in the dictionary
            data[dataiterator]={}
            #each data point contains forces, so we set the following key to 1:
            data[dataiterator]["has_forces"]=np.array([1]).reshape(-1)
            
            #this iterator will tell us the line number we are in
            iterator=-1
    
            #iterate over all lines in the output
            for line in file:
                iterator+=1
                # Getting energies
                if line.startswith("! 1 Hamiltonian Matrix"):
                    energies = np.zeros((nsinglets+ntriplets,1))
                    for istate in range(nsinglets+ntriplets):
                        energyline = file[iterator+2+istate].split()
                        energies[istate]=float(energyline[2*istate])
                    data[dataiterator]["energy"]=energies.reshape(-1)
                    
                    #get spin-orbit couplings
                    if soc_flag == True:
                        #triangular minus diagonal (times two due to complex values)
                        socs = np.zeros((int(nstates*nstates-nstates)))
                        sociterator = -1
                        for istate in range(nstates*2):
                            socsline = file[iterator+2+istate].split()
                            
                            for jstate in range(2+istate*2,2*nstates):
                                sociterator+=1
                                socs[sociterator]=float(socsline[jstate])
                        data[dataiterator]["socs"] = socs
                    
                # Getting forces
                if line.startswith("! 3 Gradient Vectors "):
                    forces = np.zeros((nsinglets+ntriplets,natoms,3))
                    for istate in range(nsinglets+ntriplets):
                        for iatom in range(natoms):
                            forceline = file[iterator+2+istate+istate*natoms+iatom].split()
                            for xyz in range(3):
                                forces[istate][iatom][xyz]=float(forceline[xyz])
                    data[dataiterator]["forces"]=forces
                
                # Getting nonadiabatic couplings (NACs)
                # Only take the ones that are between different states, so in this case: skip NAC(1,1)=0
                # Note that only one NAC is saved as NAC(1,2)=-NAC(2,1) - numbers in brackets denote states
                if line.startswith("! 5 Non-adiabatic couplings"):
                    nacs = np.zeros((nnacs,natoms,3))
                    naciterator=-1
                    for inac1 in range(nstates):
                        for inac2 in range(nstates):
                            if inac1==inac2  or inac2<inac1 or (inac1 < nsinglets and inac2>=nsinglets) or inac2>=(nsinglets+ntriplets) or inac1 >=(nsinglets+ntriplets):
                                pass
                            else:
                                naciterator+=1
                                for iatom in range(natoms):
                                    nacline = file[iterator+2+inac1*nstates+inac1*natoms*nstates+inac2*natoms+inac2+iatom].split()
                                    for xyz in range(3):
                                        nacs[naciterator][iatom][xyz] = float(nacline[xyz])
                    data[dataiterator]["nacs"]=nacs
                
                # Getting values for dipole moments
                # Transition and permanent dipole moments are included in x,y,z direction (last dimension)
                # The values are read line after line starting from the diagonal values
                # In the QM.out file, there are 3 blocks, one block belongs to one direction (x,y,z)
                # SchNarc has ( ndipoles x (xyz) ) as a shape
                # Again we only take one diagonal of the matrix and only real values (imaginary values are 0)
                if line.startswith("! 2 Dipole Moment Matrices "):
                    dipoles = np.zeros((ndipoles,3))
                    for xyz in range(3):
                        dipoleiterator=-1
                        for idipole1 in range(nstates):
                            for idipole2 in range(nstates):
                                if idipole2<idipole1 or (idipole1 < (nsinglets) and idipole2>=nsinglets) or idipole2>=(nsinglets+ntriplets) or idipole1 >=(nsinglets+ntriplets):
                                    pass
                                else:
                                    dipoleiterator+=1
                                    dipoleline = file[iterator+2+xyz+xyz*nstates+idipole1].split()
                                    dipoles[dipoleiterator][xyz] = float(dipoleline[idipole2*2])
                    data[dataiterator]["dipoles"]=dipoles

            # convert the geometries into bohr
            atoms.append(ase.atoms.Atoms(geoms[i].get_atomic_numbers(),geoms[i].get_positions()/Bohr))
            
    return data, atoms
        
        


First, use the data extractor from SHARC to generate the output.xyz files with the geometries.
Read the geometries with ase.
We have chosen to take the last and the last-5 geometry for the adaption of the training set

## Get geometries 
### Trajectories from ML model trained on energies and forces (Hessian approximation)

In [23]:
geoms=[]
ntraj = 10
ind=[]
for i in range(ntraj):
    
    #generate output.xyz 
    os.system("cd %s/TRAJ_%i/ ; %s/data_extractor_NetCDF.x -xyz output.dat; cd ../.."%(path,i+1,SHARC))
    
    geometries = ase.io.read("%s/TRAJ_%i/output.xyz"%(path,i+1),":")
    
    #this happens if the first geometry is predicted with a too large error
    if len(geometries)==0:
        if i+1 not in ind:
            ind.append(i+1)
    else:
        #append geometries with last geometry of trajectory file
        #and with last -5
        geoms.append(geometries[-1])
        geoms.append(geometries[-5])
print(len(geoms))


20


## Make folders for calculation

In this example we will generate X folder (X=number of geometries for expanding the training set).
We will copy all relevant files to make QM calculations with SHARC.
Make sure you have downloaded the "Inputfile" folder and make sure to adapt the "run.sh" file for your cluster. 
For the sake of the tutorial, we will also provide the reference calculations in the zip-file "AdaptiveSampling-Calculations.zip". So you don't need to run the calculations, but you can copy the results and go on with adapting the training set.
Note that this zip only contains all file in the first folder, and in all other folders only the inputs and outputs are saved.

Note that you additionally have to copy the SAVEDIR folder and to adapt the "QMstring"-file to carry out phasecorrection as it was done for generating the initial training set.

In [24]:
#make adaptive sampling folders
inputs = "/../../Inputfiles/"
for i in range(len(geoms)):
    os.system("mkdir QM")
    os.system("mkdir QM/Geom_%05d"%i)
    ase.io.write("QM/Geom_%05d/geometry.xyz"%i,geoms[i])
    os.system("cp QM/Geom_%05d/geometry.xyz QM/Geom_%05d/QM.in"%(i,i))
    os.system("cat %s/QMstring >> QM/Geom_%05d/QM.in" %(inputs,i))
    os.system(" cp %s/MOLPRO* QM/Geom_%05d/"%(inputs,i))
    #os.system(" mkdir QM/Geom_%05d/SAVEDIR"%i)
    #os.system(" cp %s/wf.1 QM/Geom_%05d/SAVEDIR"%(inputs,i))
    os.system(" cp %s/run.sh QM/Geom_%05d/" %(inputs,i))


In [25]:
print("Copy the submission script for the cluster you use into every folder and submit the calculations")

Copy the submission script for the cluster you use into every folder and submit the calculations


## Adapting the training set

For now we copy a dummy-QMout file from the Initial training set generation into every QM-folder for demonstration.
Note that units are all given in a.u. and that the geometries need to be saved in a.u. as well

In [26]:
for i in range(len(geoms)):
    os.system("cp ../Initial/ICOND_00001/QM.out QM/Geom_%05d/"%i)


## Read QMout

In [27]:
#iterating over all files
nfiles = len(geoms)
#define number of states
ntriplets = 0
nsinglets = 2
nstates = nsinglets + 3 * ntriplets

#define number of atoms
natoms = 12
#number of nacs
nnacs = int(nsinglets*(nsinglets-1)/2) + int(ntriplets*(ntriplets-1)/2)
#number of dipole moment values
ndipoles = int(nsinglets+ntriplets+nnacs)


#for conversion of atoms into bohr
from ase.units import Bohr

#data dictionary for updating the data base
data = {}

path="./QM/Geom"
#we don't have spin-orbit couplings
socs=False
#read properties
data,atoms=read_QMout(data,geoms,nfiles,natoms,nnacs,ndipoles,nstates,path,socs,nsinglets,ntriplets)





### Add data to the data base

In [28]:
# load the old data base
from ase.db import connect
old_db = connect("../../DBs/Fulvene.db")
for i in range(len(data)):
    old_db.write(atoms[i],data=data[i])

In [31]:
print("We have added",len(atoms), "data points to the data set and now have a total number of",len(old_db), "data points.")

We have added 20 data points to the data set and now have a total number of 120 data points.


Now retrain the models and redo the sampling until the training set is large enough.
For fulvene and the 40 fs we compute, the training set is large enough.

If you want to make production runs and use a second model to compare the energies, but not to compute forces or hessians, then use " --emodel2 PATH/ " instead of "--adaptive --modelpaths PATH".
The threshold is set to 1 eV per default. You can change the threshold via "--thresholds 1 1 1 1 1". (thresholds are always given for every possible property).